# Problem Solution
This script will attempt to solve the problem explained in the `TODO.md` file. It will be focused only in the first section, "Objetivo 1: lectura y tratamiento".


## Step 1: Download the data
First we will try to download the raw data from 2018.

In [21]:
import numpy as np
import pandas as pd
from datetime import datetime

In [2]:
url = r'https://datos.madrid.es/egob/catalogo/201410-7775096-calidad-aire-diario.csv'
# sep specifies the column delimiter
df = pd.read_csv(url, sep=';')
print(df.head())


   PROVINCIA  MUNICIPIO  ESTACION  MAGNITUD PUNTO_MUESTREO   ANO  MES  D01  \
0         28         79         4         1  28079004_1_38  2018    1  1.0   
1         28         79         4         1  28079004_1_38  2018    2  5.0   
2         28         79         4         1  28079004_1_38  2018    3  1.0   
3         28         79         4         1  28079004_1_38  2018    4  2.0   
4         28         79         4         1  28079004_1_38  2018    5  2.0   

  V01  D02  ...  D27  V27  D28  V28  D29  V29  D30  V30  D31  V31  
0   V  1.0  ...  2.0    V  2.0    V  6.0    V  5.0    V  6.0    V  
1   V  3.0  ...  3.0    V  2.0    V  0.0    N  0.0    N  0.0    N  
2   V  2.0  ...  2.0    V  4.0    V  2.0    V  2.0    V  2.0    V  
3   V  3.0  ...  1.0    V  2.0    V  2.0    V  2.0    V  0.0    N  
4   V  2.0  ...  2.0    V  2.0    V  3.0    V  2.0    V  2.0    V  

[5 rows x 69 columns]


## Step 2: Format the data
Once we have all the raw data in a `dataframe` we can focus on the transformations for making it's use easier.

### 2.1 Remove unwanted data
We will remove all data that is not related with NO2, We only care about all the data with Magnitud = 08. Once the data is removed, the column no longer makes sense.

In [3]:
# Getting our MAGNITUD and Removing the column MAGNITUD:
nitrogen_dioxide_df = df.loc[df['MAGNITUD'] == 8]

In [4]:
nitrogen_dioxide_df = nitrogen_dioxide_df.drop(columns=['MAGNITUD'])

In [5]:
print(nitrogen_dioxide_df.head())

    PROVINCIA  MUNICIPIO  ESTACION PUNTO_MUESTREO   ANO  MES   D01 V01   D02  \
36         28         79         4   28079004_8_8  2018    1  21.0   V  29.0   
37         28         79         4   28079004_8_8  2018    2  58.0   V  47.0   
38         28         79         4   28079004_8_8  2018    3  35.0   V  53.0   
39         28         79         4   28079004_8_8  2018    4  38.0   V  40.0   
40         28         79         4   28079004_8_8  2018    5  25.0   V  29.0   

   V02  ...   D27 V27   D28 V28   D29 V29   D30 V30   D31 V31  
36   V  ...  26.0   V  22.0   V  67.0   V  61.0   V  74.0   V  
37   V  ...  55.0   V  52.0   V   0.0   N   0.0   N   0.0   N  
38   V  ...  40.0   V  41.0   V  14.0   V   9.0   V  16.0   V  
39   V  ...  26.0   V  14.0   V  13.0   V  24.0   V   0.0   N  
40   V  ...  26.0   V  32.0   V  47.0   V  35.0   V  31.0   V  

[5 rows x 68 columns]


We will also remove the V columns. We can verify each column and mark as NaN all the values that are not verified. After this, V column will be redundant so we will remove it.

The first step will be to define a function that can do this:

In [6]:
def verify_measure(measures, verifications):
    """
     Checks if a given measurement is verified. 
     It builds a new column were all values not verified are change to NaN
    Args:
        measures: The raw measures that might be verified or not
        verifications:  The verification data.
        A letter 'V' means verified and 'N' means NOT verified

    Returns:
        The corrected column with only verified data and NaNs 
        in non verified measurements.

    """
    corrected = measures.copy()
    for index, measure in measures.iteritems():
        if 'V' in verifications[index]:
            corrected[index] = measure
        elif 'N' in verifications[index]:
            corrected[index] = np.nan
        else:
            print("A measure was neither verified nor unverified.")

    return corrected

To use the function we will need all the measurements and verifications isolated and a df to edit:

In [7]:
d_df = nitrogen_dioxide_df[nitrogen_dioxide_df.columns[
    pd.Series(nitrogen_dioxide_df.columns).str.startswith('D')]]
v_df = nitrogen_dioxide_df[nitrogen_dioxide_df.columns[
    pd.Series(nitrogen_dioxide_df.columns).str.startswith('V')]]

print("Sample value that we now should change from 0, N to NaN")
print(d_df['D29'][37])
print(v_df['V29'][37])

Sample value that we now should change from 0, N to NaN
0.0
N


In [8]:
# Create a new dataframe to edit (avoiding warning):
corrected_measures = d_df.copy()

Now we can proceed with the measurement correction:

In [9]:
for column_d, column_v in zip(d_df.columns, v_df.columns):
    corrected_measures[column_d] = verify_measure(d_df[column_d], v_df[column_v])

In [10]:
print("If this has worked now 'corrected_measures['D29'][37]' should be NaN: ")
print(corrected_measures['D29'][37])

If this has worked now 'corrected_measures['D29'][37]' should be NaN: 
nan


The next step is to build our final df without v and with verified measurements:

In [11]:
no2_no_v_df = nitrogen_dioxide_df.drop(columns=(list(d_df.columns) + list(v_df.columns)))
no2_no_v_df = pd.concat([no2_no_v_df, corrected_measures], axis=1)

print(no2_no_v_df.head())

    PROVINCIA  MUNICIPIO  ESTACION PUNTO_MUESTREO   ANO  MES   D01   D02  \
36         28         79         4   28079004_8_8  2018    1  21.0  29.0   
37         28         79         4   28079004_8_8  2018    2  58.0  47.0   
38         28         79         4   28079004_8_8  2018    3  35.0  53.0   
39         28         79         4   28079004_8_8  2018    4  38.0  40.0   
40         28         79         4   28079004_8_8  2018    5  25.0  29.0   

     D03   D04  ...   D22   D23   D24   D25   D26   D27   D28   D29   D30  \
36  35.0  26.0  ...  54.0  57.0  62.0  46.0  38.0  26.0  22.0  67.0  61.0   
37  41.0  31.0  ...  54.0  60.0  63.0  66.0  76.0  55.0  52.0   NaN   NaN   
38  27.0  25.0  ...  48.0  51.0  14.0  25.0  50.0  40.0  41.0  14.0   9.0   
39  24.0  21.0  ...  46.0  50.0  53.0  56.0  45.0  26.0  14.0  13.0  24.0   
40  42.0  41.0  ...  49.0  45.0  41.0  38.0  28.0  26.0  32.0  47.0  35.0   

     D31  
36  74.0  
37   NaN  
38  16.0  
39   NaN  
40  31.0  

[5 rows x 37 

We are done! We have removed all the obviously redundant information! 🙂

## 2.2 The not that obvious formatting.

By looking at the data I have multiple ideas of how to process it or make it more usefull but I think at this point I would talk to the people that will be using the data to get more info on **how** they will be using it.

Coding this might be time consuming, I will be explaining it instead. If I have some extra time after the other sections I will tackle it and see how far I can get.

### Examples
1. Clearly we need some kind of timestamp to make the processing much easier. I would change ANO, MES and D* columns for only one timestamp column using [rfc3339](https://tools.ietf.org/html/rfc3339) format. In real life choosing a timestamp format is not that easy, some situations I have seen:
 * Multiple timestamp formats at the same time in one application because they were through a transition.
 * Systems using timestamps in String format (they said it increased simple query speed but I was not that sure...) requiring a conversion to a date/time format for complex queries.
 * Systems using Local time instead of UTC, in this cases the client thinks that is executing a task at a certain moment but the server might be logging that it happend an hour later, ...
2. The column PUNTO_MUESTREO is redundant but we don't know how we should show the weather station info. If It was my decision I would choose one of the following options:
 * Substitute the numbers of PROVINCIA and MUNICIPIO for its names, leave the stations as numbers and remove the PUNTO_MUESTRO column. This would simplify the querying process and save time looking at the tables with that info for every query.
 * Get all GPS locations of each station and update ESTACION with this coordinates. Then I would remove PROVINCIA MUNICIPIO and PUNTO_MUESTREO. Probably if you want to draw an interactive map it is easier with GPS locations.
3. There is also a general problem with how the info is organized. It is not natural because there are a lot of columns and some NaN values could be avoided. For example February 31 will always be NaN for all stations.

If I took care of all this formatting on my own, the data would be organized like this:

| TimeStamp            | WeatherStation    | No2  |
| -------------------- | ----------------- | ---- |
| 2018-01-01T08:00:00Z | +48.52+002.20     | 21.0 |
| 2018-01-02T08:00:00Z | +48.8577+002.295  | 29.0 |
| 2018-01-03T08:00:00Z | +40.75-074.00     | 35.0 |
| 2018-01-04T08:00:00Z | +40.6894-074.0447 | 26.0 |
| 2018-01-05T08:00:00Z | +40.4165-3.70256  | 54.0 |

If we wanted it to also be easier to use we could create another table to store data about the Weather stations and with a simple join you could search No2 values based on MUNICIPIO for example.



### Deeper explanation of Example 3

Altering the whole table structure is going to take a few steps I will try to describe them in detail:
1. Creating the destination table:

In [24]:
# First we create a table with all the possible dates for a measurement:
dates = pd.date_range(start='1/1/2018', periods=365, freq='D',
                      name='date').to_frame()
dates.head()
# Then a table with all possible pollution stations and it's data:
station_data = no2_no_v_df[
    ['PROVINCIA', 'MUNICIPIO', 'ESTACION', 'PUNTO_MUESTREO']].drop_duplicates()
station_data.head()
# Now we have to join them because we will have one daily measurement from each stations.
# We cannot simply merge de dataframes becuase they do not have a matching column.
# Since the key of the resulting dataframe will be the combination of the keys 
# of both original dataframes we can simply create a temporary column and do the join
dates['tmp'] = 1
station_data['tmp'] = 1
new_format_df = pd.merge(dates, station_data, on='tmp').drop('tmp', axis=1)
# To finish the destination table we will simply add 
# a no2 column for the nitorgen dioxide values with all nans (for the moment)
new_format_df['no2'] = np.nan
# Using a 30 for the head might be more enlightening
# new_format_df.head(30)
new_format_df.head()


,date,PROVINCIA,MUNICIPIO,ESTACION,PUNTO_MUESTREO,no2
0,2018-01-01,28,79,4,28079004_8_8,NaN
1,2018-01-01,28,79,8,28079008_8_8,NaN
2,2018-01-01,28,79,11,28079011_8_8,NaN
3,2018-01-01,28,79,16,28079016_8_8,NaN
4,2018-01-01,28,79,17,28079017_8_8,NaN


2. Populate destination table with original table's data:

In [36]:
for row in new_format_df.itertuples():
    # Define the key variables in each row:
    year = row.date.year
    month = row.date.month
    day = row.date.day
    day_formatted = 'D' + f'{day:02}'
    
    # Extract relevant value from origin table:
    # We obtain the day column and filter by 'year', 'month' and 'PUNTO_MUESTREO'
    no2_value = no2_no_v_df[day_formatted][
        (no2_no_v_df['PUNTO_MUESTREO'] == row.PUNTO_MUESTREO) &
        (no2_no_v_df['MES'] == month) &
        (no2_no_v_df['ANO'] == year)].item()
    
    # Write relevant data retrieved from origin table to destination table:
    # we write the 'no2' column from the row with that timestamp and that 'PUNTO_MUESTREO'
    new_format_df.loc[
            (new_format_df['date'] == datetime(year=year, month=month, day=day)) &
            (new_format_df['PUNTO_MUESTREO'] == row.PUNTO_MUESTREO)
            , 'no2'] = no2_value
    
new_format_df.head()

,date,PROVINCIA,MUNICIPIO,ESTACION,PUNTO_MUESTREO,no2
0,2018-01-01,28,79,4,28079004_8_8,21.0
1,2018-01-01,28,79,8,28079008_8_8,47.0
2,2018-01-01,28,79,11,28079011_8_8,24.0
3,2018-01-01,28,79,16,28079016_8_8,31.0
4,2018-01-01,28,79,17,28079017_8_8,22.0


## Step 3: Store the data

It seems we will be having a lot of rows and a relatively simple data strcuture. I would clearly go for a SQL approach, we do not need the flexibility of a NonSQL database such as Mongo. We need to be able to store the data as efficiently as possible and I know from experience that **PostgreSQL** is easy to use, quick and efficient.

The problem that we might have with **PostgreSQL** is that it is not natively distributed so it is not that good working in a lot of nodes simultaneously. As a consequence **PostgreSQL** cannot scale horizontally, having a huge database on the cloud might be expensive.

For the moment I would go with **PostgreSQL** but always keeping in mind those problems.